In [17]:
import torch
from torchvision.datasets import FashionMNIST
fm_train = FashionMNIST(root='.', train=True, download=True) # 훈련세트
fm_test = FashionMNIST(root='.', train=False, download=True) # 테스트 세트

In [18]:
type(fm_train.data)

torch.Tensor

In [19]:
# 훈련세트 ,테스트 세드 갯수
print(fm_train.data.shape, fm_test.data.shape)
print(fm_train.targets.shape, fm_test.targets.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.Size([60000]) torch.Size([10000])


In [20]:
train_input = fm_train.data
train_target = fm_train.targets

In [21]:
# 정규화 - 0~1
train_scaled = train_input / 255.0 

In [22]:
# 훈련세트, 검증 세트
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [23]:
# 층 구성
import torch.nn as nn
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

In [24]:
# 층 구성 요약 정보  - input_size=(배치 사이즈, ....)
from torchinfo import summary
summary(model, input_size=(32, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 10]                  --
├─Flatten: 1-1                           [32, 784]                 --
├─Linear: 1-2                            [32, 100]                 78,500
├─ReLU: 1-3                              [32, 100]                 --
├─Linear: 1-4                            [32, 10]                  1,010
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.54
Input size (MB): 0.10
Forward/backward pass size (MB): 0.03
Params size (MB): 0.32
Estimated Total Size (MB): 0.45

# 맥 - mps / 그래픽 가속기
```python
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
```

In [25]:
# GPU 또는 CPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # 모델에 GPU 또는 CPU 연산 설정

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=100, bias=True)
  (2): ReLU()
  (3): Linear(in_features=100, out_features=10, bias=True)
)

## 분류 손실 함수
- 이진 분류 : nn.BCELoss() : 손실 함수 + 출력함수(sigmoid)
- 다중 분류 : nn.CrossEntropyLoss() : 손실 함수 + 출력함수(softmax)

In [26]:
# 손실함수, 옵티마이저
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # 손실 함수 - Categorical Crossentropy
optimizer = optim.Adam(model.parameters())

In [30]:
# 학습
epochs = 5
batches = int(len(train_scaled) / 32) # 1500
for epoch in range(epochs):
    model.train() # 학습 시작으로 알려줌
    train_loss = 0
    for i in range(batches): # 미니배치 경사 하강법
        inputs = train_scaled[i*32:(i+1)*32].to(device)
        targets = train_target[i*32:(i+1)*32].to(device)

        optimizer.zero_grad() # 그레디언트 초기화
        outputs = model(inputs) # 추론

        loss = criterion(outputs, targets)
        loss.backward() # 역전파
        optimizer.step() # 모델 파라미터 업데이트 (가중치, 절편)
        train_loss += loss.item() # 현재 배치의 로스값을 추가
    
    print(f"에포크:{epoch+1}, 손실값: {train_loss / batches:.4f}")

에포크:1, 손실값: 0.2970
에포크:2, 손실값: 0.2811
에포크:3, 손실값: 0.2685
에포크:4, 손실값: 0.2577
에포크:5, 손실값: 0.2475


In [42]:
model.eval() # 모델 평가
with torch.no_grad(): # 평가시에는 그레디언트 연산 X
    val_scaled = val_scaled.to(device)
    val_target = val_target.to(device)
    outputs = model(val_scaled)
    predicts = torch.argmax(outputs, 1)
    corrects = (predicts == val_target).sum().item() # True은 1로 연산, 정답 갯수
    accuracy = corrects / len(val_target)
    print(f"정확도:{accuracy:.4f}")

정확도:0.8832
